In [1]:
import os
os.environ["PYTHONHASHSEED"] = "0"
os.environ["TF_DETERMINISTIC_OPS"] = "1"     # deterministic GPU ops
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"   # deterministic cuDNN kernels

import tensorflow as tf
from tensorflow import keras
from keras import layers
from pathlib import Path
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input

# Set the seeds for reproducibility
import random
SEED = 2648509283
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

/Users/eugeneshevchenko/Courses/RD_Computer-Vision/HW/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [46]:
# DATASET_DIR = "/content/gdrive/MyDrive/Courses/Robot_Dreams/CV/FinalProject/data-original/chest_xray_240"
DATASET_DIR = "../data/chest_xray"
DATASET_PATH = Path(DATASET_DIR)
TRAIN_PATH = DATASET_PATH / "train"
TEST_PATH = DATASET_PATH / "test"
CLASSES = ["NORMAL", "PNEUMONIA"]
ID_TO_CLASS = {0:'NORMAL', 1:'PNEUMONIA'}
IMG_SIZE_SIDE = 224
IMG_SIZE = (IMG_SIZE_SIDE, IMG_SIZE_SIDE)
BATCH = 32
VAL_FRACTION = 0.15
PAD_TO_ASPECT_RATIO=False
AUTOTUNE = tf.data.AUTOTUNE

In [63]:
train_ds, val_ds = keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    class_names=CLASSES,
    label_mode='binary',
    color_mode='grayscale',
    image_size=IMG_SIZE,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=PAD_TO_ASPECT_RATIO,
    batch_size=BATCH,
    validation_split=VAL_FRACTION,
    subset='both',
    shuffle=True,
    seed=SEED,
)

Found 5216 files belonging to 2 classes.
Using 4434 files for training.
Using 782 files for validation.


In [61]:
test_ds = keras.preprocessing.image_dataset_from_directory(
    TEST_PATH,
    labels='inferred',
    class_names=CLASSES,
    label_mode='binary',
    color_mode='grayscale',
    image_size=IMG_SIZE,
    pad_to_aspect_ratio=PAD_TO_ASPECT_RATIO,
    batch_size=BATCH,
    shuffle=False,
)

Found 624 files belonging to 2 classes.


In [82]:
def to_rgb_and_pp(x, y):
    if x.shape[-1] == 1:
        x = tf.image.grayscale_to_rgb(x)
    x = preprocess_input(x)   # ResNet-50 preprocessing
    return x, y

In [64]:
# aug = keras.Sequential([
# ])
# def augment(x, y): return aug(x, training=True), y
def augment(x, y): return x, y

train_ds = train_ds.map(to_rgb_and_pp, num_parallel_calls=AUTOTUNE).map(augment, AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(to_rgb_and_pp, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [8]:
base = keras.applications.ResNet50(
    include_top=False, weights="imagenet", input_shape=IMG_SIZE + (3,)
)
base.trainable = False  # Stage 1: freeze backbone

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = inputs
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
# Optional small head
x = layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
TARGET_METRIC = "val_aucpr"
class_weight = None

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.AUC(curve="PR", name="aucpr"),
        keras.metrics.AUC(curve="ROC", name="auc"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
    ],
)

callbacks = [
    keras.callbacks.ModelCheckpoint("resnet50_head.keras", monitor=TARGET_METRIC,
                                    save_best_only=True, mode="max"),
    keras.callbacks.EarlyStopping(monitor=TARGET_METRIC, mode="max", patience=6, restore_best_weights=True),
]

model.fit(train_ds, validation_data=val_ds, epochs=10, class_weight=class_weight, callbacks=callbacks)

Epoch 1/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 867s 6s/step - auc: 0.9556 - aucpr: 0.9827 - loss: 0.3533 - precision: 0.9678 - recall: 0.8686 - val_auc: 0.9966 - val_aucpr: 0.9988 - val_loss: 0.1471 - val_precision: 0.9928 - val_recall: 0.9583
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 708s 5s/step - auc: 0.9904 - aucpr: 0.9968 - loss: 0.1614 - precision: 0.9756 - recall: 0.9665 - val_auc: 0.9985 - val_aucpr: 0.9995 - val_loss: 0.0968 - val_precision: 0.9795 - val_recall: 0.9948
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 700s 5s/step - auc: 0.9903 - aucpr: 0.9967 - loss: 0.1525 - precision: 0.9679 - recall: 0.9723 - val_auc: 0.9986 - val_aucpr: 0.9995 - val_loss: 0.1014 - val_precision: 0.9929 - val_recall: 0.9688
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 730s 5s/step - auc: 0.9931 - aucpr: 0.9975 - loss: 0.1284 - precision: 0.9768 - recall: 0.9805 - val_auc: 0.9986 - val_aucpr: 0.9995 - val_loss: 0.0875 - val_precision: 0.9811 - val_recall: 0.9931
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 694s 5s/step

In [10]:
# Unfreeze top layers
for layer in base.layers:
    layer.trainable = False
for layer in base.layers:
    # unfreeze the last block
    if "conv5_block" in layer.name:
        layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.AUC(curve="PR", name="aucpr"),
        keras.metrics.AUC(curve="ROC", name="auc"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
    ],
)

callbacks_ft = [
    keras.callbacks.ModelCheckpoint("resnet50_finetune.keras", monitor=TARGET_METRIC,
                                    save_best_only=True, mode="max"),
    keras.callbacks.EarlyStopping(monitor=TARGET_METRIC, mode="max", patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor=TARGET_METRIC, mode="max", factor=0.5, patience=2, min_lr=1e-6),
]

model.fit(train_ds, validation_data=val_ds, epochs=15, class_weight=class_weight, callbacks=callbacks_ft)

Epoch 1/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 1083s 8s/step - auc: 0.9921 - aucpr: 0.9975 - loss: 0.1399 - precision: 0.9801 - recall: 0.9697 - val_auc: 0.9985 - val_aucpr: 0.9995 - val_loss: 0.0845 - val_precision: 0.9982 - val_recall: 0.9757 - learning_rate: 2.0000e-05
Epoch 2/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 1046s 8s/step - auc: 0.9993 - aucpr: 0.9998 - loss: 0.0578 - precision: 0.9954 - recall: 0.9942 - val_auc: 0.9993 - val_aucpr: 0.9998 - val_loss: 0.0667 - val_precision: 0.9948 - val_recall: 0.9896 - learning_rate: 2.0000e-05
Epoch 3/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 1036s 7s/step - auc: 0.9998 - aucpr: 0.9999 - loss: 0.0499 - precision: 0.9962 - recall: 0.9960 - val_auc: 0.9985 - val_aucpr: 0.9995 - val_loss: 0.0675 - val_precision: 0.9948 - val_recall: 0.9878 - learning_rate: 2.0000e-05
Epoch 4/15
139/139 ━━━━━━━━━━━━━━━━━━━━ 979s 7s/step - auc: 0.9999 - aucpr: 1.0000 - loss: 0.0454 - precision: 0.9975 - recall: 0.9960 - val_auc: 0.9986 - val_aucpr: 0.9996 - val_loss: 0.0681 - val_prec

In [11]:
y_val = np.concatenate([y.numpy() for _, y in val_ds])
y_val_pred = model.predict(val_ds).ravel()

best_thr, best_f1 = 0.5, -1
for thr in np.linspace(0.05, 0.95, 19):
    y_hat = (y_val_pred >= thr).astype(int)
    tp = np.sum((y_hat==1)&(y_val==1))
    fp = np.sum((y_hat==1)&(y_val==0))
    fn = np.sum((y_hat==0)&(y_val==1))
    prec = tp/(tp+fp+1e-9)
    rec  = tp/(tp+fn+1e-9)
    f1 = 2*prec*rec/(prec+rec+1e-9)
    if f1 > best_f1:
        best_f1, best_thr = f1, thr

print("Best F1:", best_f1, "at threshold:", best_thr)

25/25 ━━━━━━━━━━━━━━━━━━━━ 102s 4s/step
Best F1: 0.746040407392309 at threshold: 0.05


In [12]:
from sklearn.metrics import precision_recall_curve

target_recall = 0.94

prec, rec, thr = precision_recall_curve(y_val, y_val_pred)
prec_ = prec[1:]
rec_ = rec[1:]
thr_ = thr

mask = rec_ >= target_recall
if np.any(mask):
    sel = np.max(np.where(mask)[0])
    # sel = np.where(mask)[0][np.argmax(prec_[mask])]
else:
    # Fallback: pick the closest recall to the target (if target is unattainable)
    sel = int(np.argmin(np.abs(rec_ - target_recall)))

best_thr = float(thr_[sel])

# Compute and print metrics at this threshold
y_hat = (y_val_pred >= best_thr).astype(int)
tp = np.sum((y_hat == 1) & (y_val == 1))
fp = np.sum((y_hat == 1) & (y_val == 0))
fn = np.sum((y_hat == 0) & (y_val == 1))
precision_at = tp / (tp + fp + 1e-9)
recall_at = tp / (tp + fn + 1e-9)
f1_at = 2 * precision_at * recall_at / (precision_at + recall_at + 1e-9)

print(f"Target recall: {target_recall:.3f}")
print(f"Chosen threshold: {best_thr:.4f}")
print(f"Val Precision: {precision_at:.4f}  Recall: {recall_at:.4f}  F1: {f1_at:.4f}")


Target recall: 0.940
Chosen threshold: 0.9783
Val Precision: 0.7366  Recall: 0.6944  F1: 0.7149


### Evaluation

In [48]:
model = keras.models.load_model("../model/resnet50_finetune.keras")

In [56]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,116,165 (210.25 MB)

 Trainable params: 15,501,313 (59.13 MB)

 Non-trainable params: 8,612,224 (32.85 MB)

 Optimizer params: 31,002,628 (118.27 MB)

In [62]:
test_ds = test_ds.map(to_rgb_and_pp, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [81]:
model.evaluate(val_ds, verbose=2)

25/25 - 7s - 281ms/step - auc: 0.5000 - aucpr: 0.7366 - loss: 120.4616 - precision: 0.0000e+00 - recall: 0.0000e+00


[120.46157836914062, 0.7365728616714478, 0.5, 0.0, 0.0]

> The training process was failed in Google Colab and the model was saved in an incorrect state. So we can't evaluate it on Test Set :-(